In [1]:
#
# Test of station initialisation
#
import tango
import time
import json
import numpy as np

from ska_tango_base.commands import ResultCode
from ska_tango_base.control_model import HealthState
# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

# define devices
station = tango.DeviceProxy('low-mccs/station/001')
subrack = tango.DeviceProxy('low-mccs/subrack/0001')
t1 = tango.DeviceProxy('low-mccs/tile/0001')
# t2 = tango.DeviceProxy('low-mccs/tile/0002')
# t3 = tango.DeviceProxy('low-mccs/tile/0003')
t4 = tango.DeviceProxy('low-mccs/tile/0004')
# t5 = tango.DeviceProxy('low-mccs/tile/0005')
# t6 = tango.DeviceProxy('low-mccs/tile/0006')
# t7 = tango.DeviceProxy('low-mccs/tile/0007')
# t8 = tango.DeviceProxy('low-mccs/tile/0008')

station.logginglevel=5
devices = [station, subrack, t1, t4, ]


In [2]:
# Put everything online
for d in devices:
    d.adminmode = 0
time.sleep(0.2)
station.standby()
time.sleep(5)

In [3]:
print(station.status())
# define static attributes for initialisation
t1.logginglevel=5
station.SetBeamformerTable([128,0,1,0,0,0,0])
print(station.beamformertable.reshape(-1,7))
station.statictimedelays=list(range(32))*16
# print(station.statictimedelays.reshape(-1,32))
station.preaduLevels=list(range(32))*16
# print(station.preaduLevels.reshape(-1,32))
station.channeliserRounding=[4]*512
station.cspRounding=[5]*384
station.SetLmcDownload('{"destination_ip": "10.0.0.98", "mode": "40g"}')
#station.SetLmcIntegratedDownload('{"destination_ip": "10.0.0.98", "mode": "40g"}')
station.SetCspIngest('{"destination_ip": "10.0.0.98"}')
time.sleep(0.1)
t1.logginglevel=3

The device is in STANDBY state.
[[128   0   1   0   0   0   0]]


In [15]:
# turn station on
t1.logginglevel=5
t4.logginglevel=5
station.on()
state = station.tileprogrammingstate
tm = 0
init = False
for t in range (30):
    time.sleep(2)
    tm = tm + 2
    print(f't={tm}: subrack: {subrack.status()}')
    if subrack.state() == tango._tango.DevState.ON:
        break
for t in range(30):
    time.sleep(2)
    tm = tm + 2
    s_new = station.tileprogrammingstate
    if s_new != state:
        print(f't={tm}: state = {s_new}')
        state = s_new
    if all(s == 'Initialised' for s in state):
        init = True
        break
if init:
    print(f't={tm}: Station initialized')
else:
    print(f't={tm}: Timeout during intialisation')
time.sleep(3)
t1.logginglevel=3
t4.logginglevel=3

t=2: subrack: The device is in ON state.
t=16: state = ('NotProgrammed', 'NotProgrammed')
t=24: state = ('Programmed', 'Programmed')
t=44: state = ('Initialised', 'Initialised')
t=44: Station initialized


In [16]:
t1.logginglevel=5
start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+4), RFC_FORMAT)
station.StartAcquisition(json.dumps({
  "start_time": start_time}))
print(f'Tile time: {t1.fpgatime} - Sync time: {start_time}')
print(f'Programmed Sync time: {t1.fpgareferencetime}')
time.sleep(1)
# check that synchronization worked
for t in range (1):
    tm1 = datetime.strftime(datetime.fromtimestamp(time.time()), RFC_FORMAT)
    tm2 = t1.fpgatime
    tm3 = t1.fpgaframetime
    print(f'time:{tm1} pps time:{tm2} frame time:{tm3}')
    #time.sleep(2)
for i in range(30):
    tm = tm + 1
    cur_time=int(t1.readregister('fpga1.pps_manager.curr_time_read_val')[0])
    start_time=int(t1.readregister('fpga1.pps_manager.sync_time_val')[0])
    print(f'Current: {cur_time} - Start: {start_time} difference: {cur_time-start_time} frame time:{t1.fpgaframetime}')
    if cur_time > start_time:
        break
    time.sleep(1)

tm1 = datetime.strftime(datetime.fromtimestamp(time.time()), RFC_FORMAT)
tm2 = t1.fpgatime
tm3 = t1.fpgaframetime
print(f'time:{tm1} pps time:{tm2} frame time:{tm3}')
print(f't={tm}: state = {station.tileprogrammingstate}')
t1.logginglevel=3

Tile time: 2023-02-26T22:08:30.000000Z - Sync time: 2023-02-26T22:08:34.000000Z
Programmed Sync time: 2106-02-07T06:28:15.000000Z
time:2023-02-26T22:08:31.895669Z pps time:2023-02-26T22:08:31.000000Z frame time:2106-02-07T06:28:15.000000Z
Current: 1677449311 - Start: 1677449314 difference: -3 frame time:2106-02-07T06:28:15.000000Z
Current: 1677449312 - Start: 1677449314 difference: -2 frame time:2106-02-07T06:28:15.000000Z
Current: 1677449313 - Start: 1677449314 difference: -1 frame time:2106-02-07T06:28:15.000000Z
Current: 1677449314 - Start: 1677449314 difference: 0 frame time:2106-02-07T06:28:15.765573Z
Current: 1677449315 - Start: 1677449314 difference: 1 frame time:2106-02-07T06:28:16.781361Z
time:2023-02-26T22:08:35.975348Z pps time:2023-02-26T22:08:35.000000Z frame time:2106-02-07T06:28:16.783296Z
t=49: state = ('Synchronised', 'Synchronised')


In [17]:
# Show chainig of tile IPs
core1_config = json.loads(t1.get40gcoreconfiguration(json.dumps({'core_id': 0, 'arp_table_entry': 0})))
core2_config = json.loads(t1.get40gcoreconfiguration(json.dumps({'core_id': 1, 'arp_table_entry': 0})))
print(f'Configuration of beamformer 40g ports')
print(f"TPM 1: {core1_config['source_ip']}:{core1_config['source_port']} -> "
      f"{core1_config['destination_ip']}:{core1_config['destination_port']}"
      f"   MAC:{hex(core1_config['source_mac'])}")
print(f"       {core2_config['source_ip']}:{core2_config['source_port']} -> "
      f"{core2_config['destination_ip']}:{core2_config['destination_port']}"
      f"   MAC:{hex(core2_config['source_mac'])}")
core1_config = json.loads(t4.get40gcoreconfiguration(json.dumps({'core_id': 0, 'arp_table_entry': 0})))
core2_config = json.loads(t4.get40gcoreconfiguration(json.dumps({'core_id': 1, 'arp_table_entry': 0})))
print(f"TPM 4: {core1_config['source_ip']}:{core1_config['source_port']} -> "
      f"{core1_config['destination_ip']}:{core1_config['destination_port']}"
      f"   MAC:{hex(core1_config['source_mac'])}")
print(f"       {core2_config['source_ip']}:{core2_config['source_port']} -> "
      f"{core2_config['destination_ip']}:{core2_config['destination_port']}"
      f"   MAC:{hex(core2_config['source_mac'])}")


Configuration of beamformer 40g ports
TPM 1: 10.0.0.152:61648 -> 10.0.0.154:4660   MAC:0x62000a000098
       10.0.0.153:61648 -> 10.0.0.155:4660   MAC:0x62000a000099
TPM 4: 10.0.0.154:61648 -> 10.0.0.98:4660   MAC:0x62000a00009a
       10.0.0.155:61648 -> 10.0.0.98:4660   MAC:0x62000a00009b


In [18]:
# Show LMC IPs 
core1_config = json.loads(t1.get40gcoreconfiguration(json.dumps({'core_id': 0, 'arp_table_entry': 1})))
core2_config = json.loads(t1.get40gcoreconfiguration(json.dumps({'core_id': 1, 'arp_table_entry': 1})))
print('Configuration of LMC 40G port')
print(f"TPM1: Core 1: {core1_config['source_ip']}:{core1_config['source_port']} -> "
      f"{core1_config['destination_ip']}:{core1_config['destination_port']}")
print(f"      Core 2: {core2_config['source_ip']}:{core2_config['source_port']} -> "
      f"{core2_config['destination_ip']}:{core2_config['destination_port']}")
core1_config = json.loads(t4.get40gcoreconfiguration(json.dumps({'core_id': 0, 'arp_table_entry': 1})))
core2_config = json.loads(t4.get40gcoreconfiguration(json.dumps({'core_id': 1, 'arp_table_entry': 1})))
print(f"TPM4: Core 1: {core1_config['source_ip']}:{core1_config['source_port']} -> "
      f"{core1_config['destination_ip']}:{core1_config['destination_port']}")
print(f"      Core 2: {core2_config['source_ip']}:{core2_config['source_port']} -> "
      f"{core2_config['destination_ip']}:{core2_config['destination_port']}")

Configuration of LMC 40G port
TPM1: Core 1: 10.0.0.152:61648 -> 10.0.0.98:4660
      Core 2: 10.0.0.153:61648 -> 10.0.0.98:4660
TPM4: Core 1: 10.0.0.154:61648 -> 10.0.0.98:4660
      Core 2: 10.0.0.155:61648 -> 10.0.0.98:4660


In [19]:
print(f'Beamformer table: {t1.beamformertable.reshape(-1,7)}')
print(f'Static delays: {t1.statictimedelays.reshape(-1,32)}')
print(f'Preadu levels: {t1.preaduLevels.reshape(-1,32)}')
print(f'Channeliser rounding: {t1.channeliserRounding[0]}')
print(f'CSP rounding: {t4.cspRounding[0]}')
print(f'ARP table: {t1.getArpTable()}')

Beamformer table: [[128   0   1   0   0   0   0]]
Static delays: [[ 0.    1.25  2.5   2.5   3.75  5.    6.25  7.5   7.5   8.75 10.   11.25
  12.5  12.5  13.75 15.   16.25 17.5  17.5  18.75 20.   21.25 22.5  22.5
  23.75 25.   26.25 27.5  27.5  28.75 30.   31.25]]
Preadu levels: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Channeliser rounding: 3
CSP rounding: 5
ARP table: {"0": [0, 1], "1": [0, 1]}


In [23]:
station.SendDataSamples(json.dumps({'data_type': 'raw'}))

[array([0], dtype=int32), ['SendDataSamples command completed OK']]

In [24]:
station.SendDataSamples(json.dumps({'data_type': 'channel'}))

[array([0], dtype=int32), ['SendDataSamples command completed OK']]

In [25]:
station.standby()

[array([2], dtype=int32), ['1677450372.0711076_116821448308244_Standby']]